In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import $ivy.`org.apache.spark::spark-mllib:2.4.0`
import $ivy.`sh.almond::ammonite-spark:0.4.0`
import $ivy.`org.datasyslab:geospark:1.2.0`

import org.apache.spark.serializer.KryoSerializer
import org.apache.spark.storage.StorageLevel
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.rdd.RDD
import org.datasyslab.geospark.enums.{GridType, IndexType}
import org.datasyslab.geospark.spatialOperator.JoinQuery
import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader
import scala.collection.JavaConverters._
import java.io._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = AmmoniteSparkSession.builder()
    .master("local[*]").appName("Validator")
    .getOrCreate()
import spark.implicits._
val appID = spark.sparkContext.applicationId

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                    

import $ivy.$                                

import $ivy.$                              


import org.apache.spark.serializer.KryoSerializer

import org.apache.spark.storage.StorageLevel

import org.apache.spark.mllib.evaluation.RegressionMetrics

import org.apache.spark.rdd.RDD

import org.datasyslab.geospark.enums.{GridType, IndexType}

import org.datasyslab.geospark.spatialOperator.JoinQuery

import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader

import scala.collection.JavaConverters._

import java.io._

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@5f9476d7
import spark.implicits._

appID: String = "local-1557626873550"

### Collecting datasets...
Polygons from 18 states were collected for both source and target in WKT format. They are available at: https://github.com/aocalderon/RIDIR/tree/master/Datasets/AreaTablesValidation.

In [2]:
import sys.process._

val path = "/home/acald013/RIDIR/Datasets/AreaTablesValidation"
s"ls -lah ${path}" #| "grep wkt" !

-rw-rw-r-- 1 acald013 acald013 7.3M Apr 19 09:29 AL_source.wkt
-rw-rw-r-- 1 acald013 acald013 3.9M Apr 19 09:29 AL_target.wkt
-rw-rw-r-- 1 acald013 acald013 5.4M Apr 19 09:29 AZ_source.wkt
-rw-rw-r-- 1 acald013 acald013 3.4M Apr 19 09:29 AZ_target.wkt
-rw-rw-r-- 1 acald013 acald013 5.0M Apr 19 09:29 CO_source.wkt
-rw-rw-r-- 1 acald013 acald013 2.8M Apr 19 09:29 CO_target.wkt
-rw-rw-r-- 1 acald013 acald013 1.5M Apr 19 09:29 CT_source.wkt
-rw-rw-r-- 1 acald013 acald013 1.4M Apr 19 09:29 CT_target.wkt
-rw-rw-r-- 1 acald013 acald013 9.0M Apr 19 09:29 GA_source.wkt
-rw-rw-r-- 1 acald013 acald013 4.8M Apr 19 09:29 GA_target.wkt
-rw-rw-r-- 1 acald013 acald013 3.2M Apr 19 09:29 IL_source.wkt
-rw-rw-r-- 1 acald013 acald013 1.8M Apr 19 09:29 IL_target.wkt
-rw-rw-r-- 1 acald013 acald013 3.3M Apr 19 09:29 IN_source.wkt
-rw-rw-r-- 1 acald013 acald013 2.3M Apr 19 09:29 IN_target.wkt
-rw-rw-r-- 1 acald013 acald013 6.9M Apr 19 09:29 LA_source.wkt
-rw-rw-r-- 1 acald013 acald013 3.0M Apr 19 09:29 LA_tar

import sys.process._


path: String = "/home/acald013/RIDIR/Datasets/AreaTablesValidation"
res1_2: Int = 0

For each set of source & target, we run the corresponding script:

* For GeoPandas implementation: https://github.com/aocalderon/RIDIR/blob/master/Code/Validation/GeoPandas_area_table_tester.py
* For GeoSpark implementation: https://github.com/aocalderon/RIDIR/blob/master/Code/Areal/src/main/scala/GeoSpark_area_table_tester.scala

Each script save the results to disk for further analysis (files are also available in the same repo).

In [3]:
val path = "/home/acald013/RIDIR/Datasets/AreaTablesValidation"
s"ls -lah ${path}" #| "grep tsv" !

-rw-rw-r-- 1 acald013 acald013 162K Apr 19 09:29 AL_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 157K Apr 19 09:29 AL_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 165K Apr 19 09:29 AZ_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 161K Apr 19 09:29 AZ_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 169K Apr 19 09:29 CO_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 164K Apr 19 09:29 CO_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 145K Apr 19 09:29 CT_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 141K Apr 19 09:29 CT_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 257K Apr 19 09:29 GA_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 251K Apr 19 09:29 GA_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 126K Apr 19 09:29 IL_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 123K Apr 19 09:29 IL_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 186K Apr 19 09:29 IN_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 181K Apr 19 09:29 IN_geospark_test.tsv
-rw-rw-r-- 1 

path: String = "/home/acald013/RIDIR/Datasets/AreaTablesValidation"
res2_1: Int = 0

### Set a particular state to run the validation...

In [4]:
val state = "TX"

state: String = "TX"

### Reading results from geopandas implementation...

In [5]:
val geopandas = spark.read.option("header", "false").option("delimiter", "\t").csv(s"${path}/${state}_geopandas_test.tsv").distinct()
geopandas.count()

geopandas: Dataset[Row] = [_c0: string, _c1: string ... 1 more field]
res4_1: Long = 21410L

### Reading results from geospark implementation...

In [6]:
val geospark = spark.read.option("header", "false").option("delimiter", "\t").csv(s"${path}/${state}_geospark_test.tsv").distinct()
geospark.count()

geospark: Dataset[Row] = [_c0: string, _c1: string ... 1 more field]
res5_1: Long = 21410L

### Merging both result sets...

In [7]:
val p = geopandas.map(p => (p.getString(0), p.getString(1), p.getString(2).toDouble)).rdd
            .sortBy(p => (p._2, p._1, p._3)).repartition(1)
p.toDF().show(false)
p.count()

+------+------+---------------------+
|_1    |_2    |_3                   |
+------+------+---------------------+
|T54482|S61320|1.4856713197347244E-5|
|T54483|S61320|1.3428845503735151E-5|
|T54488|S61320|1.5692278894226508E-5|
|T54490|S61320|2.015843729787822E-5 |
|T54894|S61320|6.340132701726107E-5 |
|T54915|S61320|1.010519144095369E-6 |
|T54916|S61320|0.047360737737180074 |
|T54917|S61320|0.027117068771871727 |
|T54918|S61320|2.556941751028444E-5 |
|T54923|S61320|1.2113117883442573E-5|
|T54916|S61321|2.3144935039285635E-6|
|T54918|S61321|0.0015869187196737413|
|T54923|S61321|1.481265382233853E-5 |
|T54894|S61322|1.3310058456443617E-5|
|T54900|S61322|2.6295495066915202E-5|
|T54916|S61322|8.33440464527646E-6  |
|T54917|S61322|2.510670359317722E-5 |
|T54918|S61322|0.0070110029427875534|
|T54923|S61322|6.754176895075E-5    |
|T54354|S61323|6.167120075582282E-6 |
+------+------+---------------------+
only showing top 20 rows



p: RDD[(String, String, Double)] = MapPartitionsRDD[52] at repartition at cmd6.sc:2
res6_2: Long = 21410L

In [8]:
val s = geospark.map(s => (s.getString(0), s.getString(1), s.getString(2).toDouble)).rdd
            .sortBy(p => (p._1, p._2, p._3)).repartition(1)
s.toDF().show(false)
s.count()

+------+------+---------------------+
|_1    |_2    |_3                   |
+------+------+---------------------+
|S61320|T54482|1.4856713197347244E-5|
|S61320|T54483|1.3428845503735151E-5|
|S61320|T54488|1.5692278894226508E-5|
|S61320|T54490|2.015843729787822E-5 |
|S61320|T54894|6.34013270172611E-5  |
|S61320|T54915|1.010519144095369E-6 |
|S61320|T54916|0.047360737737180074 |
|S61320|T54917|0.027117068771871727 |
|S61320|T54918|2.5569417510284444E-5|
|S61320|T54923|1.2113117883442573E-5|
|S61321|T54916|2.3144935039285635E-6|
|S61321|T54918|0.0015869187196737413|
|S61321|T54923|1.481265382233853E-5 |
|S61322|T54894|1.3310058456443617E-5|
|S61322|T54900|2.6295495066915202E-5|
|S61322|T54916|8.33440464527646E-6  |
|S61322|T54917|2.5106703593177223E-5|
|S61322|T54918|0.0070110029427875534|
|S61322|T54923|6.754176895075E-5    |
|S61323|T54354|6.167120075582282E-6 |
+------+------+---------------------+
only showing top 20 rows



s: RDD[(String, String, Double)] = MapPartitionsRDD[73] at repartition at cmd7.sc:2
res7_2: Long = 21410L

In [9]:
val areas = p.zip(s)
areas.toDF().show(false)

+---------------------------------------+---------------------------------------+
|_1                                     |_2                                     |
+---------------------------------------+---------------------------------------+
|[T54482, S61320, 1.4856713197347244E-5]|[S61320, T54482, 1.4856713197347244E-5]|
|[T54483, S61320, 1.3428845503735151E-5]|[S61320, T54483, 1.3428845503735151E-5]|
|[T54488, S61320, 1.5692278894226508E-5]|[S61320, T54488, 1.5692278894226508E-5]|
|[T54490, S61320, 2.015843729787822E-5] |[S61320, T54490, 2.015843729787822E-5] |
|[T54894, S61320, 6.340132701726107E-5] |[S61320, T54894, 6.34013270172611E-5]  |
|[T54915, S61320, 1.010519144095369E-6] |[S61320, T54915, 1.010519144095369E-6] |
|[T54916, S61320, 0.047360737737180074] |[S61320, T54916, 0.047360737737180074] |
|[T54917, S61320, 0.027117068771871727] |[S61320, T54917, 0.027117068771871727] |
|[T54918, S61320, 2.556941751028444E-5] |[S61320, T54918, 2.5569417510284444E-5]|
|[T54923, S61320

areas: RDD[((String, String, Double), (String, String, Double))] = ZippedPartitionsRDD2[78] at zip at cmd8.sc:1

In [10]:
import org.apache.spark.sql.functions._
areas.map(a => (a._1._3, a._2._3)).toDF("area1", "area2").withColumn("diff", abs($"area1" - $"area2")).orderBy(desc("diff")).show(50, truncate = false)

+---------------------+---------------------+----------------------+
|area1                |area2                |diff                  |
+---------------------+---------------------+----------------------+
|0.0014175904472804926|0.0014175904472804907|1.951563910473908E-18 |
|0.0010438371736246278|0.0010438371736246276|2.1684043449710089E-19|
|0.001723702869320858 |0.0017237028693208583|2.1684043449710089E-19|
|0.0015346975919289106|0.0015346975919289104|2.1684043449710089E-19|
|2.8575885729881326E-4|2.857588572988131E-4 |1.6263032587282567E-19|
|8.516409768940811E-4 |8.516409768940812E-4 |1.0842021724855044E-19|
|5.778733057945471E-4 |5.77873305794547E-4  |1.0842021724855044E-19|
|3.0489059025834317E-4|3.048905902583433E-4 |1.0842021724855044E-19|
|6.334366241623034E-4 |6.334366241623035E-4 |1.0842021724855044E-19|
|9.02660620372196E-4  |9.026606203721961E-4 |1.0842021724855044E-19|
|3.572639985071235E-5 |3.572639985071246E-5 |1.0842021724855044E-19|
|3.108097415731932E-4 |3.108097415

import org.apache.spark.sql.functions._


### Running some metrics to test similarity...

In [11]:
val reg = new RegressionMetrics(areas.map(a => (a._1._3, a._2._3)))
reg.r2
reg.meanAbsoluteError
reg.meanSquaredError
reg.rootMeanSquaredError

reg: RegressionMetrics = org.apache.spark.mllib.evaluation.RegressionMetrics@4cb72a64
res10_1: Double = 1.0
res10_2: Double = 5.262230073287982E-22
res10_3: Double = 1.9782160357652242E-40
res10_4: Double = 1.4064906810090225E-20